In [1]:
#Import necessary packages and load cleaned data.
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
df=pd.read_csv('cleaned_data.csv', parse_dates=['date_time'])
#Creating month column to make some inference calls simpler
df['month'] = df.date_time.dt.month

In [2]:
#DataFrame with all shooting types of targeted or indiscriminate
targ_ind = df.loc[df.shooting_type.str.contains('targeted|indiscriminate'),:]

#Dataframe with all shooting types NOT targeted or indiscriminate
other = df.loc[~df.shooting_type.str.contains('targeted|indiscriminate'),:]

#Individual dataframes for shooting types targeted/indiscriminate
targ = df.loc[df.shooting_type=='targeted',:]
ind = df.loc[df.shooting_type=='indiscriminate',:]

### Days of the Week

In my earlier data visualizations, I investigated whether there may be a relationship between the day of the week and the type of shooting that occurred.  While the days of the week looked evenly distributed for indiscriminate shootings, it did appear that there were substantially more targeted shootings on a Tuesday.  This lead me to run chi-squared goodness of fit test to determine if there was any statistically significant evidence to support these hypotheses.

In [3]:
#Check for statistical evidence that the distribution of day of the week is not evenly distributed
chisq, p = st.chisquare(df.groupby('day_of_week').nunique().uid)
print('Entire dataframe: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(targ_ind.groupby('day_of_week').nunique().uid)
print('Targeted or indiscriminate: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(targ.groupby('day_of_week').nunique().uid)
print('Targeted: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(ind.groupby('day_of_week').nunique().uid)
print('Indiscriminate: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

Entire dataframe: Chi-square = 4.75  P-value = 0.3134
Targeted or indiscriminate: Chi-square = 4.23  P-value = 0.376
Targeted: Chi-square = 5.43  P-value = 0.2461
Indiscriminate: Chi-square = 3.74  P-value = 0.4425


Suprisingly, none of the chi-square goodness of fit tests found any statistically significant evidence that the days of the week were not evenly distributed, not even when looking at the targeted shooting types in particular.

### Month of the Year

Next, I investigated whether there may be a relationship between the month of the year and the type of shooting that occurred. Again, through my data visualizations, there appeared to be a similar relationship as before: the indiscriminate shootings appeared roughly evenly distributed while the targeted shootings appeared to happen primarily in January and September/October.  This lead me to run chi-squared goodness of fit tests to determine if there was any statistically significant evidence to support these hypotheses. I chose only to include the typical school months of September through May because the low number of incidents in the summer months could influence this relationship.

In [4]:
len(ind.loc[((df.month!=6) & (df.month!=7) & (df.month!=8)),:])/9

2.3333333333333335

In [5]:
#Check for statistical evidence that the distribution of the month is not evenly distributed
chisq, p = st.chisquare(df.loc[((df.month!=6) & (df.month!=7) & (df.month!=8)),:].groupby('month').nunique().uid)
print('Entire dataframe: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(targ_ind.loc[((targ_ind.month!=6) & (targ_ind.month!=7) & (targ_ind.month!=8)),:].groupby('month').nunique().uid)
print('Targeted or indiscriminate: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(targ.loc[((targ.month!=6) & (targ.month!=7) & (targ.month!=8)),:].groupby('month').nunique().uid)
print('Targeted: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

chisq, p = st.chisquare(ind.loc[((ind.month!=6) & (ind.month!=7) & (ind.month!=8)),:].groupby('month').nunique().uid)
print('Indiscriminate: Chi-square = {}  P-value = {}'.format(round(chisq, 2), round(p, 4)))

Entire dataframe: Chi-square = 16.73  P-value = 0.0331
Targeted or indiscriminate: Chi-square = 21.56  P-value = 0.0058
Targeted: Chi-square = 23.81  P-value = 0.0025
Indiscriminate: Chi-square = 2.67  P-value = 0.8494


There is statistically significant evidence to suggest that certain months are more likely to have incidents of school shooting than other months, especially when considering the targeted shooting types.  We can also see that there isn't statistical evidence to suggest that the months are not evenly distributed for indiscriminate shootings; however, I would caution about putting too much confidence in those results as the number of indiscriminate shootings was substantially smaller than typically required of a chi-squared test.

Focusing on the targeted shootings, which months are the most likely to have an incident?

In [17]:
#Creating dictionary to relate month number to month name
month_dict = {9:'September', 10:'October', 11:'November', 12:'December', 1:'January', 2:'Februrary', 3:'March', 4:'April', 5:'May'}

#Calculate length of targeted dataframe not including summer months as well as expected proportion of shooting incidents for each month if there is no difference
n=len(targ.loc[((df.month!=6) & (df.month!=7) & (df.month!=8)),:])
exp_prop = 1/9

#Running a z-test for proportions for each month
for month in month_dict.keys():
    count = len(targ.loc[df.month == month,:])
    z, p = proportions_ztest(count=count, nobs=n, value=exp_prop)
    
    print('{}: z = {}, p-value = {}'.format(month_dict[month], round(z, 2), round(p, 4)))

September: z = 0.5, p-value = 0.6195
October: z = 1.18, p-value = 0.2375
November: z = -4.82, p-value = 0.0
December: z = -1.65, p-value = 0.0985
January: z = 2.57, p-value = 0.0102
Februrary: z = 0.1, p-value = 0.9232
March: z = 0.5, p-value = 0.6195
April: z = -4.82, p-value = 0.0
May: z = -0.93, p-value = 0.3549


Interestingly enough, there is very strong statistical evidence to suggest that the incident rate of targeted school shootings are substantially lower than expected in the November and April, and there is relatively strong statistical evidence to suggest that the incident rate of targeted school shootings is higher than expected in January.  It's interesting that months with significant holidays appear to have lower targeted school shooting rates, while January (the month after several significant holidays) appears to have a higher rate.

Disclaimer: Because of the small sample size of shootings in each month, these statistics (while interesting) may not be 100% relaible.

### County Characteristics

I merged a variety of county characteristics with the school shooter dataset from the Washington Post.  I am interested if there are any substantial differences between these county characteristics for the different groupings of shooting types.

In [7]:
#Numerical columns of interest to perform independent t-tests for a difference in means.
ttest_col = ['mentally_unhealthy_days', 'excessive_drinking', 'teen_birth_rate', 'uninsured', 'grad_rate', 'some_college', 'unemployed', 'children_in_poverty', 'single_parent_households', 'pcp_rate']

In [19]:
#Comparing county characteristics for targeted/indiscriminate against the remaining shooting types
for col in ttest_col:
    t,p = st.ttest_ind(other[col], targ_ind[col])
    print('{}: t={}, p-value={}'.format(col,round(t,2),round(p,4)))

mentally_unhealthy_days: t=0.48, p-value=0.629
excessive_drinking: t=1.27, p-value=0.2087
teen_birth_rate: t=-0.12, p-value=0.9037
uninsured: t=-0.17, p-value=0.863
grad_rate: t=-0.6, p-value=0.5515
some_college: t=-0.38, p-value=0.7081
unemployed: t=0.56, p-value=0.5776
children_in_poverty: t=0.58, p-value=0.5605
single_parent_households: t=1.4, p-value=0.1644
pcp_rate: t=-1.01, p-value=0.316


It appears there are no statistically significant differences for the county characteristics when comparing the targeted/indiscriminate against the remaining types of school shootings.

In [21]:
#Comparing county characteristics for targeted vs indiscriminate
for col in ttest_col:
    t,p = st.ttest_ind(targ[col], ind[col])
    print('{}: t={}, p-value={}'.format(col,round(t,2), round(p,4)))

mentally_unhealthy_days: t=0.81, p-value=0.4214
excessive_drinking: t=-0.25, p-value=0.8033
teen_birth_rate: t=2.21, p-value=0.0303
uninsured: t=2.52, p-value=0.0138
grad_rate: t=-0.24, p-value=0.8087
some_college: t=0.11, p-value=0.9114
unemployed: t=1.76, p-value=0.0817
children_in_poverty: t=3.08, p-value=0.0029
single_parent_households: t=1.9, p-value=0.0609
pcp_rate: t=0.46, p-value=0.6476


However, when we compare targeted against indiscriminate shooting types specifically, there is some statistical evidence to suggest there are differences between the counties those occured in.  The most substantial differences between them seem to be in the teen birth rate, the rate of people uninsured and the number of children in poverty.  The counties with targeted school shootings all seem to substantially higher in regards to these characteristics.

But what about the incidents with a substantial number of casualties?

In [28]:
a = df.loc[df.casualties > 1, :]
b = df.loc[df.casualties <= 1, :]

for col in ttest_col:
    t,p = st.ttest_ind(a[col], b[col])
    print('{}: t={}, p-value={}'.format(col,round(t,2), round(p,4)))

mentally_unhealthy_days: t=-0.05, p-value=0.9635
excessive_drinking: t=2.78, p-value=0.0064
teen_birth_rate: t=-1.33, p-value=0.1852
uninsured: t=-0.78, p-value=0.4374
grad_rate: t=0.89, p-value=0.3747
some_college: t=0.43, p-value=0.6708
unemployed: t=-0.37, p-value=0.7122
children_in_poverty: t=-1.95, p-value=0.0544
single_parent_households: t=-1.21, p-value=0.2279
pcp_rate: t=-0.39, p-value=0.6957


It appears that the only statistically significant difference between counties with school shootings that had more than one casualty and ones that did not is in the excessive drinking rate (self-reported).  As seen in this sample, counties with larger casualty rates had a substantially higher excessive drinking rate than other counties.  And yet there didn't appear to be a difference in the other variables that many people would attribute to an increase in drinking: mentally unhealthy days, unemployment, etc.